## Mini Project 3 Part A

## SQL Movie Database for Movie Popularity Analysis

## 100 Points (will be converted to 10 points)

In this mini-project, you will work with a movie dataset collected by James Gaskin, available at: https://data.world/jamesgaskin/is201movies/workspace/file?filename=MovieList+New.xlsx.

Please load the pre-built SQL database by typing the following command in the terminal:

```bash
mysql -e 'CREATE DATABASE [Your_NetID]_movies;'
mysql [Your_NetID]_movies < movie_user_rating.sql
```

Replace [Your_NetID] with your NetID. For example, if your NetID is `hz333`, you would type:

```bash
mysql -e 'CREATE DATABASE hz333_movies;'
mysql hz333_movies < movie_user_rating.sql
```

Make sure you have the `movie_user_rating.sql` file in the same directory where you are running the above commands.

In the pre-built SQL database, the movie information has been loaded as an SQL table named **movie**. Additionally, randomly generated user information is available in an SQL table named **user**, and user ratings are stored in an SQL table named **rating**.

Your task is to formulate SQL queries in the Jupyter Notebook under the section "Queries to Movies SQL Database" by replacing the line "__Your_SQL_Query_Here__".

The goal of this mini-project is to analyze the movie dataset to understand the popularity of movies based on movie release_date, genre, and user ratings. The obtained results are presented as SQL Tables.

This goal is achieved by solving the following tasks:

- [Find movies titles with certain prefixes](#task-1-find-movie-titles-that-begin-with-the-sort-by-release-date-limit-to-the-latest-10-movies)
- [Find the number of movies released by years](#task-2-find-the-number-of-movies-released-by-years)
- [Identify the number of movies released by genres](#task-3-identify-the-number-of-movies-released-by-genres)
- [Identify the top rated movies by users](#task-4-identify-the-top-rated-movies-by-users)

### Imports and Definitions of helper functions

Please make sure you run the next cell to import the necessary libraries before you start working on the assigned tasks.

In [32]:
from mysql.connector import connect, Error
import time

Please make sure you run the next two cells to define the helper functions before you start working on the assigned tasks.

In [33]:
WAIT_TIME = 10
def connect_to_db(db_name):
    """
    Connect to the database
    IN: db_name, str, the name of the database in MySQL
    OUT: mydb, the database connection
    EXCEPTION: TimeoutError, if the database is not running
    """

    num_of_tries = 0
    while num_of_tries < WAIT_TIME:
        try:
            mydb = connect(host="127.0.0.1", user="root", db=db_name)
            if mydb.is_connected():
                print("Connected")
                break
        except Error as e:
            print(e)

        num_of_tries += 1
        print("Waiting for MySQL to start")
        time.sleep(1)

    if num_of_tries >= WAIT_TIME:
        raise TimeoutError("MySQL is not running")

    return mydb

In [34]:
def do_query(cursor, query):
    """
    Execute a query and print the results
    IN: cursor, the cursor object
        query, the query to be executed
    OUT: res, list[tuple], the results of the query
    """
    cursor.execute(query)
    res = cursor.fetchall()
    print(cursor.column_names)
    for row in res:
        print(row)
    return res

### Connect to Movies SQL Database

To connect to the SQL database, you need to run the following cell. Please make sure you specify your database name correctly.

In [35]:
if __name__ == "__main__":
    # all code under this block will be executed only when you directly run this Jupyter notebook
    mydb = connect_to_db("jc3003_movies") # change the database name to your database
    cursor = mydb.cursor()

Connected


### Movies SQL Database Schema

Before you start working on the assigned tasks, it is a good idea to understand the schema of the SQL database by running the next two cells.

In [36]:
def get_table_specification_query(table_name):
    """
    Generate the SQL query to show the table specification
    IN: table_name, str, the name of the table
    OUT: str, the SQL query
    """
    return f"DESC {table_name}"

In [37]:
if __name__ == "__main__":
    # all code under this block will be executed only when you directly run this Jupyter notebook
    for table_name in ['movie', 'user', 'rating']:
        print(f'Table: {table_name}')
        query = get_table_specification_query(table_name)
        res = do_query(cursor, query)

Table: movie
('Field', 'Type', 'Null', 'Key', 'Default', 'Extra')
('id', 'int', 'NO', 'PRI', None, 'auto_increment')
('title', 'varchar(100)', 'NO', 'MUL', None, '')
('release_date', 'date', 'NO', '', None, '')
('genre', 'varchar(20)', 'NO', '', None, '')
Table: user
('Field', 'Type', 'Null', 'Key', 'Default', 'Extra')
('id', 'int', 'NO', 'PRI', None, 'auto_increment')
('uname', 'char(8)', 'NO', 'UNI', None, '')
('age', 'tinyint', 'NO', '', None, '')
('state', 'varchar(14)', 'NO', '', None, '')
Table: rating
('Field', 'Type', 'Null', 'Key', 'Default', 'Extra')
('userid', 'int', 'NO', 'PRI', None, '')
('movieid', 'int', 'NO', 'PRI', None, '')
('rating', 'tinyint', 'NO', '', None, '')


### Sample Data in Movies SQL Database

Before you start working on the assigned tasks, it is a good idea to take a look at a snapshot of the data in the SQL database by running the next two cells.

In [38]:
def get_sample_data_query(table_name, num_rows=5):
    """
    Generate the SQL query to show the sample data
    IN: table_name, str, the name of the table
    OUT: str, the SQL query
    """
    return f"SELECT * FROM {table_name} LIMIT {num_rows}"

In [39]:
if __name__ == "__main__":
    # all code under this block will be executed only when you directly run this Jupyter notebook
    for table_name in ['movie', 'user', 'rating']:
        print(f'Table: {table_name}')
        query = get_sample_data_query(table_name)
        res = do_query(cursor, query)

Table: movie
('id', 'title', 'release_date', 'genre')
(1, "Look Who's Talking", datetime.date(1989, 10, 12), 'Romance')
(2, 'Driving Miss Daisy', datetime.date(1989, 12, 13), 'Comedy')
(3, 'Turner & Hooch', datetime.date(1989, 7, 28), 'Crime')
(4, 'Born on the Fourth of July', datetime.date(1989, 12, 20), 'War')
(5, 'Field of Dreams', datetime.date(1989, 4, 21), 'Drama')
Table: user
('id', 'uname', 'age', 'state')
(1, 'ZqkCe579', 19, 'Delaware')
(2, 'ktFAM702', 53, 'North Carolina')
(3, 'NeOEI446', 35, 'Delaware')
(4, 'DvXQj522', 68, 'Ohio')
(5, 'OVnAt417', 31, 'Ohio')
Table: rating
('userid', 'movieid', 'rating')
(1, 1, 7)
(1, 21, 3)
(1, 27, 3)
(1, 63, 9)
(1, 67, 4)


### Queries to Movies SQL Database

Please replace the line "__Your_SQL_Query_Here__" with your SQL query to solve the assigned tasks.

#### Task 1: Find movie titles that begin with 'The', sort by release date in descending order, and limit to the latest 10 movies

Your query result Table should look like this:

```text
+-------+--------------+
| title | release_date |
+-------+--------------+
| ...   | ...          |
+-------+--------------+
```

In [40]:
def get_latest_movies_by_title_prefix_query(prefix="The", num_rows=10):
    """
    Generate the SQL query to show the latest movies that start with a prefix
    IN: prefix, str, the prefix of the movie title
        num_rows, int, the number of rows to return
    OUT: str, the SQL query
    """
    
    return f"""
    SELECT 
        title, 
        release_date
    FROM movie
    WHERE title LIKE '{prefix}%'
    ORDER BY release_date DESC
    LIMIT {int(num_rows)};
    """

In [41]:
if __name__ == "__main__":
    # all code under this block will be executed only when you directly run this Jupyter notebook
    query = get_latest_movies_by_title_prefix_query(prefix="A", num_rows=15)
    res = do_query(cursor, query)

('title', 'release_date')
('Aladdin', datetime.date(2019, 5, 8))
('Avengers: Endgame', datetime.date(2019, 4, 22))
('Aquaman', datetime.date(2018, 11, 26))
('A Star is Born', datetime.date(2018, 8, 31))
('Ant-Man and the Wasp', datetime.date(2018, 6, 25))
('Avengers: Infinity War', datetime.date(2018, 4, 23))
('A Quiet Place', datetime.date(2018, 3, 9))
('Ant-Man', datetime.date(2015, 7, 14))
('Avengers: Age of Ultron', datetime.date(2015, 4, 22))
('American Sniper', datetime.date(2014, 12, 11))
('American Hustle', datetime.date(2013, 12, 12))
('Alvin and the Chipmunks: Chipwrecked', datetime.date(2011, 12, 14))
('Alice in Wonderland', datetime.date(2010, 3, 3))
('Alvin and the Chipmunks: The Squeakquel', datetime.date(2009, 12, 21))
('Avatar', datetime.date(2009, 12, 10))


#### Task 2: Find the number of movies released by years

##### Task 2.1: Identify the number of movies released in 2015

Your query result Table should look like this:

```text
+------------+
| num_movies |
+------------+
| ...        |
+------------+
```


Hint: `YEAR(date)` SQL function can be used to extract year from a `date` field.

For example:
```sql
SELECT YEAR(movie.release_date) FROM movie;
```

In [42]:
def get_num_movies_released_in_year_query(year=2015):
    """
    Generate the SQL query to show the number of movies released in a year
    IN: year, int, the year
    OUT: str, the SQL query
    """
    
    return f"""
    SELECT
        COUNT(*) AS num_movies
    FROM movie
    WHERE YEAR(release_date) = {year};
    """

In [43]:
if __name__ == "__main__":
    # all code under this block will be executed only when you directly run this Jupyter notebook
    query = get_num_movies_released_in_year_query()
    res = do_query(cursor, query)

('num_movies',)
(20,)


##### Task 2.2: Identify how many movies were released from 2001 to 2005 (inclusive).

Your query result Table should look like this:

```text
+------------+
| num_movies |
+------------+
| ...        |
+------------+
```

In [44]:
def get_num_movies_released_between_years_query(start_year=2001, end_year=2005):
    """
    Generate the SQL query to show the number of movies released in a interval of years
    IN: start_year, int, the start year
        end_year, int, the end year
    OUT: str, the SQL query
    """
    
    return f"""
    SELECT
        COUNT(*) AS num_movies
    FROM movie
    WHERE YEAR(release_date) >= {start_year} AND YEAR(release_date) <= {end_year};
    """

In [45]:
if __name__ == "__main__":
    # all code under this block will be executed only when you directly run this Jupyter notebook
    query = get_num_movies_released_between_years_query()
    res = do_query(cursor, query)

('num_movies',)
(100,)


#### Task 3: Identify the number of movies released by genres

##### Task 3.1: Identify all the movie genres. Sort the genres in alphabetical order.

Your query result Table should look like this:

```text
+-------+
| genre |
+-------+
| ...   |
+-------+
```

In [46]:
def get_all_movie_genres_query():
    """
    Generate the SQL query to show all the movie genres in alphabetical order
    OUT: str, the SQL query
    """
    
    return f"""
    SELECT 
        DISTINCT genre
    FROM movie
    ORDER BY genre ASC;
    """

In [47]:
if __name__ == "__main__":
    # all code under this block will be executed only when you directly run this Jupyter notebook
    query = get_all_movie_genres_query()
    res = do_query(cursor, query)

('genre',)
('Action',)
('Adventure',)
('Animation',)
('Comedy',)
('Crime',)
('Drama',)
('Family',)
('Fantasy',)
('History',)
('Horror',)
('Mystery',)
('Romance',)
('Science Fiction',)
('Thriller',)
('War',)
('Western',)


##### Task 3.2: Identify the top 3 genres with the most number of movies released in 2010.

Your query result Table should look like this:

```text
+-------+-------------+
| genre | num_moivies |
+-------+-------------+
| ...   | ...         |
+-------+-------------+
```

Note: break ties by sorting genres in alphabetical order

In [48]:
def get_top_movie_genres_in_year_query(year=2010, num_genres=3):
    """
    Generate the SQL query to show the top genres with the most number of movies released in a year
    IN: year, int, the year
        num_genres, int, the number of genres to return
    OUT: str, the SQL query
    """
    
    return f"""
    SELECT
        genre,
        COUNT(*) AS num_movies
    FROM movie
    WHERE YEAR(release_date) = {year}
    GROUP BY genre
    ORDER BY COUNT(title) DESC
    LIMIT {num_genres};
    """

In [49]:
if __name__ == "__main__":
    # all code under this block will be executed only when you directly run this Jupyter notebook
    query = get_top_movie_genres_in_year_query()
    res = do_query(cursor, query)

('genre', 'num_movies')
('Animation', 6)
('Action', 3)
('Drama', 2)


#### Task 4: Identify the top rated movies by users

##### Task 4.1: Identify the top 5 highest rated movies of 2017. Keep one decimal place for the computation of average rating.

Your query result Table should look like this:

```text
+-------+------------+
| title | avg_rating |
+-------+------------+
| ...   | ...        |
+-------+------------+
```

Hint: Use `ROUND(num, decimal_places)` function to round off the average rating to one decimal place.

For example:
```sql
SELECT ROUND(AVG(rating.rating, 1) FROM rating;
```

In [50]:
def get_top_movies_in_year_query(year=2017, num_movies=5):
    """
    Generate the SQL query to show the top movies of a year
    IN: year, int, the year
        num_movies, int, the number of movies to return
    OUT: str, the SQL query
    """
    
    return f"""
    SELECT
        title,
        ROUND(AVG(rating.rating), 1) AS avg_rating
    FROM movie
    JOIN rating ON movie.id = rating.movieid
    WHERE YEAR(movie.release_date) = {year}
    GROUP BY title
    ORDER BY avg_rating DESC
    LIMIT {num_movies};
    """

In [51]:
if __name__ == "__main__":
    # all code under this block will be executed only when you directly run this Jupyter notebook
    query = get_top_movies_in_year_query()
    res = do_query(cursor, query)

('title', 'avg_rating')
('Beauty and the Beast', Decimal('5.8'))
('Dunkirk', Decimal('5.8'))
('The LEGO Batman Movie', Decimal('5.7'))
('It', Decimal('5.6'))
('Spider-Man: Homecoming', Decimal('5.6'))


#### Task 4.2: Identify the users who have an average rating of at least 8.0 for movies in the years 2010-2015 (inclusive). Sort the users by their name. Keep one decimal place for the computation of average rating.

Your query result Table should look like this:

```text
+----------+------------+
| uname    | avg_rating |
+----------+------------+
| ...      | ...        |
+----------+------------+
```

In [52]:
def get_users_with_high_average_rating_between_years_query(start_year=2010, end_year=2015, min_avg_rating=8.0):
    """
    Generate the SQL query to show the users with high average rating in a range of years
    IN: start_year, int, the start year
        end_year, int, the end year
        min_avg_rating, float, the minimum average rating
    OUT: str, the SQL query
    """
    
    return f"""
    SELECT
        user.uname,
        ROUND(AVG(rating.rating), 1) AS avg_rating
    FROM rating
    JOIN user ON rating.userid = user.id
    JOIN movie ON rating.movieid = movie.id
    WHERE YEAR(movie.release_date) >= {start_year} AND YEAR(movie.release_date) <= {end_year}
    GROUP BY user.uname
    HAVING ROUND(AVG(rating.rating), 1) >= {min_avg_rating}
    ORDER BY uname;
    """

In [53]:
if __name__ == "__main__":
    # all code under this block will be executed only when you directly run this Jupyter notebook
    query = get_users_with_high_average_rating_between_years_query()
    res = do_query(cursor, query)

('uname', 'avg_rating')
('bdZWY381', Decimal('8.2'))
('DQtdi052', Decimal('8.0'))
('ePuFV640', Decimal('8.0'))
('haBkx803', Decimal('8.1'))
('QSJUb581', Decimal('8.2'))
('TdMcx810', Decimal('8.0'))
('tvdUE823', Decimal('8.3'))
('vEraH377', Decimal('8.1'))
('VLUPZ650', Decimal('8.3'))
('vqXGj062', Decimal('8.0'))
('yVIBT711', Decimal('8.4'))


#### Task 4.3: Identify the ratings given by the user `yVIBT711` for movies released in the years 2010-2015 (inclusive). Sort the movies by their release date in ascending order.

Your query result Table should look like this:

```text
+-------+--------+--------------+
| title | rating | release_date |
+-------+--------+--------------+
| ...   | ...    | ...          |
+-------+--------+--------------+
```

Note: break ties by movie title in ascending order.

In [54]:
def get_user_ratings_for_movies_between_years_query(user_name='yVIBT711', start_year=2010, end_year=2015):
    """
    Generate the SQL query to show the ratings given by a user for movies released in a range of years
    IN: user_name, str, the user name
        start_year, int, the start year
        end_year, int, the end year
    OUT: str, the SQL query
    """
    
    return f"""
    SELECT
        movie.title,
        rating.rating,
        movie.release_date
    FROM rating
    JOIN movie ON rating.movieid = movie.id
    JOIN user ON rating.userid = user.id
    WHERE YEAR(movie.release_date) >= {start_year} AND YEAR(movie.release_date) <= {end_year} AND user.uname = '{user_name}'
    ORDER BY movie.release_date ASC;
    """

In [55]:
if __name__ == "__main__":
    # all code under this block will be executed only when you directly run this Jupyter notebook
    query = get_user_ratings_for_movies_between_years_query()
    res = do_query(cursor, query)

('title', 'rating', 'release_date')
('The Twilight Saga: Eclipse', 6, datetime.date(2010, 6, 23))
("Madagascar 3: Europe's Most Wanted", 10, datetime.date(2012, 6, 6))
('The Twilight Saga: Breaking Dawn - Part 2', 10, datetime.date(2012, 11, 13))
('The Croods', 5, datetime.date(2013, 3, 20))
('The Amazing Spider-Man 2', 9, datetime.date(2014, 4, 16))
('Godzilla', 10, datetime.date(2014, 5, 14))
('Interstellar', 9, datetime.date(2014, 11, 5))


#### Task 4.4: Identify the movies released in the same year as the movie `Wonder Woman` and have a higher average rating than `Wonder Woman`. Sort the movies by their average rating in descending order. Keep one decimal place for the computation of average rating.

Your query result Table should look like this:

```text
+-------+------------+
| title | avg_rating |
+-------+------------+
| ...   | ...        |
+-------+------------+
```

Note: break ties by movie title in ascending order.

In [56]:
def get_movies_higher_rating_than_movie_query(movie_title='Wonder Woman'):
    """
    Generate the SQL query to show the movies released in the same year as a movie and have a higher average rating
    IN: movie_title, str, the movie title
    OUT: str, the SQL query
    """
    
    return f"""
    SELECT 
        movie.title,
        ROUND(AVG(rating.rating), 1) AS avg_rating
    FROM rating
    JOIN movie ON rating.movieid = movie.id
    WHERE 
        YEAR(movie.release_date) = (
            SELECT 
                YEAR(release_date)
            FROM movie
            WHERE title = '{movie_title}'
        )
    GROUP BY movie.title
    HAVING 
        avg_rating > (
            SELECT 
                ROUND(AVG(rating.rating), 1)
            FROM rating
            JOIN movie ON rating.movieid = movie.id
            WHERE title = '{movie_title}'
    )
    ORDER BY avg_rating DESC;
    """

In [57]:
if __name__ == "__main__":
    # all code under this block will be executed only when you directly run this Jupyter notebook
    query = get_movies_higher_rating_than_movie_query()
    res = do_query(cursor, query)

('title', 'avg_rating')
('Beauty and the Beast', Decimal('5.8'))
('Dunkirk', Decimal('5.8'))
('The LEGO Batman Movie', Decimal('5.7'))
('It', Decimal('5.6'))
('Spider-Man: Homecoming', Decimal('5.6'))
('Thor: Ragnarok', Decimal('5.6'))
('Coco', Decimal('5.5'))
('Guardians of the Galaxy Vol. 2', Decimal('5.5'))
('Jumanji: Welcome to the Jungle', Decimal('5.5'))
('Justice League', Decimal('5.5'))
('Logan', Decimal('5.5'))
('The Greatest Showman', Decimal('5.5'))
('Get Out', Decimal('5.4'))
('Kong: Skull Island', Decimal('5.4'))
('The Fate of the Furious', Decimal('5.4'))
('Despicable Me 3', Decimal('5.3'))


### Note: Do not forget to close the connection to the database.

In [58]:
if __name__ == "__main__":
    # all code under this block will be executed only when you directly run this Jupyter notebook
    cursor.close()
    mydb.close()